In [14]:
import pandas as pd
import os
from glob import glob

import re

In [11]:
RU_WEBS_DIR = "./../a-crawling/crawling/crawling/spiders/pages/ru"
EN_WEBS_DIR = "./../a-crawling/crawling/crawling/spiders/pages/en"

ru_webs_pl = glob(f"{RU_WEBS_DIR}/**.html")
en_webs_pl = glob(f"{EN_WEBS_DIR}/**.html")

print(f"Russian websites: {len(ru_webs_pl)}")
print(f"English websites: {len(en_webs_pl)}")

Russian websites: 195
English websites: 565


In [12]:
RU_LEMMAS_FILE = "./../b-tokens/ru_lemmas.txt"
EN_LEMMAS_FILE = "./../b-tokens/en_lemmas.txt"

RU_TOKENS_FILE = "./../b-tokens/ru_tokens.txt"
EN_TOKENS_FILE = "./../b-tokens/en_tokens.txt"

In [54]:
# config

lang = "RU" # RU or EN
webs_pl = ru_webs_pl
lemmas_file = RU_LEMMAS_FILE

In [63]:
# lemma - key, list of tokens - value
lemmas_map = dict()

# index. lemma - key, list of file indexes - value
index = dict()

In [64]:
file = open(lemmas_file, 'r')

lines = file.readlines()
for line in lines:
    tokens = re.split('\\s+', line)
    
    lemma_dirty = tokens[0]
    lemma = lemma_dirty[:len(lemma_dirty)-1]
    
    lemmas_map[lemma] = []
    for i in range(1, len(tokens) - 1):
        token = tokens[i]
        if (not len(token.strip()) == 0):
            lemmas_map[lemma].append(token)

file.close()

lemmas_map

{'ответить': ['ответили', 'ответит', 'ответила', 'ответить'],
 'наука': ['науке', 'науки', 'наук', 'наука', 'наукой', 'науку'],
 'использовать': ['использовали',
  'использует',
  'используемое',
  'использую',
  'использована',
  'используйте',
  'используем',
  'используемых',
  'использовано',
  'используете',
  'использовала',
  'используя',
  'используй',
  'использовать',
  'используют'],
 'хороший': ['лучший',
  'лучшая',
  'хорошим',
  'хорошими',
  'хорошему',
  'наилучшего',
  'хороший',
  'хорошая',
  'хорош',
  'лучшего',
  'лучшим',
  'хороших',
  'лучше',
  'лучшие',
  'хороша',
  'хорошего',
  'лучших',
  'хорошие',
  'хорошей',
  'лучшее',
  'хорошее',
  'хорошем'],
 'принимать': ['принимая',
  'принимать',
  'принимает',
  'принимают',
  'принимающих',
  'принимаемых',
  'принимаете',
  'принимаешь'],
 'витамин': ['витамином',
  'витаминов',
  'витамины',
  'витаминах',
  'витаминами',
  'витамина',
  'витамин'],
 'автор': ['автора', 'автор', 'авторов', 'авторы', 'авто

In [48]:
import html2text
h2t = html2text.HTML2Text()
h2t.ignore_links = True

In [57]:
import spacy
import spacy_transformers

nlp_en = spacy.load("en_core_web_sm")
nlp_ru = spacy.load("ru_core_news_sm")

def get_tokens(text):
    if (lang == "RU"):
        doc = nlp_ru(text)
    else:
        doc = nlp_en(text)
    
    return [w for w in doc if (w.is_alpha and not w.is_stop)]

In [65]:
class InvertedIndexEntry:
    def __init__(self, lemma):
        self.lemma = lemma
        self.count = 0
        self.files = []
        
    def add_count(self, count):
        self.count += count

    def add_file(self, file_idx):
        self.files.append(file_idx)

In [62]:
for pl in webs_pl[:1]:
    with open(pl, "r") as file:
        file_content = file.read()
        file_text = h2t.handle(file_content)
        
        file_tokens = get_tokens(file_text)

        for token in file_tokens[:5]:
            lemma = token.lemma_
            print(lemma)

            # знакомая лемма - уже есть в списке detected лемм
            if (lemma in lemmas_map.keys()):

                if lemma not in index.keys():
                    index[lemma] = InvertedIndexEntry(lemma)
                
        

top
mail
na
разбор
использовать


In [ ]:
# write to index file